In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_6/models/B5R2b5_AN_1FC_fold6_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold6.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(658, 25)
(658, 25)
Normal:  (429, 25)
Abnormal:  (229, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,133,133,13,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,103.0,132.0,675.0,590.0,0.108818,0.146881,0.707317,0.657370,AB01 P4-2 C013.JPG,AB01 P4-2 C013.JPG
1,1,1,134,134,13,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,283.0,142.0,384.0,399.0,0.294227,0.158833,0.398510,0.444223,AB01 P2 C013.JPG,AB01 P2 C013.JPG
2,2,2,135,135,13,P5,P51,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,434.0,530.0,0.248619,0.113017,0.445672,0.589641,AB01 P5-1 C013.JPG,AB01 P5-1 C013.JPG
3,3,3,136,136,13,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,274.0,159.0,432.0,415.0,0.287594,0.176762,0.453008,0.462151,AB01 P1 C013.JPG,AB01 P1 C013.JPG
4,4,4,137,137,13,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,256.0,164.0,546.0,436.0,0.277132,0.182738,0.591085,0.486056,AB01 P4-1 C013.JPG,AB01 P4-1 C013.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 658 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.03003319, -0.16804729,  0.19052753, ..., -0.13313718,
        -0.15530434, -0.12897794],
       [-0.07203252,  0.05900411,  0.26476866, ..., -0.09191625,
        -0.02476287, -0.11975719],
       [-0.06685548, -0.09877829, -0.17445455, ...,  0.01485514,
        -0.20063858, -0.11507014],
       ...,
       [-0.07341453,  0.69406402, -0.19565792, ...,  0.72113341,
        -0.03598743,  0.72017437],
       [-0.11135938,  0.79271704, -0.04277746, ...,  0.7153371 ,
        -0.192799  ,  0.37826243],
       [-0.19293116,  0.3434312 , -0.22788349, ...,  0.07600363,
        -0.13786831,  0.301305  ]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.030033,-0.168047,0.190528,0.228052,-0.133099,-0.073283,0.147607,-0.080972,-0.119341,-0.105819,...,-0.134593,0.135659,-0.166105,-0.026275,-0.170909,-0.152570,-0.167494,-0.133137,-0.155304,-0.128978
1,-0.072033,0.059004,0.264769,0.037914,-0.195818,-0.077529,0.233249,-0.063936,-0.203762,-0.133224,...,-0.089111,-0.080490,0.077862,0.033225,-0.126178,-0.110011,-0.137647,-0.091916,-0.024763,-0.119757
2,-0.066855,-0.098778,-0.174455,-0.097675,-0.172910,-0.119162,-0.183226,-0.079950,-0.119603,-0.213992,...,0.116817,-0.005807,0.064179,-0.082199,-0.155933,-0.118044,-0.116492,0.014855,-0.200639,-0.115070
3,-0.164899,-0.137043,0.030739,0.282006,-0.110273,-0.154812,0.096774,-0.149772,-0.065819,-0.151823,...,-0.133407,-0.046785,-0.144984,-0.099277,-0.107320,-0.144773,-0.165934,0.055076,-0.071099,-0.100872
4,-0.168137,-0.054375,-0.102106,0.019492,-0.163883,-0.227633,-0.084770,-0.097355,-0.082006,-0.156650,...,-0.054416,-0.085508,-0.086097,0.067142,-0.196706,-0.237798,-0.075828,0.223701,-0.144849,-0.078928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,-0.089932,0.966502,-0.061667,-0.077552,0.749401,-0.156538,-0.115387,0.022432,0.792956,-0.158872,...,0.323928,-0.120093,0.448052,0.469421,-0.027156,-0.183324,-0.014921,0.625583,-0.066625,0.457226
654,0.041996,-0.166815,0.024205,-0.179397,-0.155007,-0.190471,-0.091025,0.342036,0.167473,-0.130458,...,0.206877,-0.103440,-0.114873,-0.115036,0.043427,-0.186619,-0.126861,0.025019,-0.139925,-0.125518
655,-0.073415,0.694064,-0.195658,-0.050105,0.957885,-0.167023,-0.168058,0.331353,0.739299,-0.216589,...,0.474633,-0.176992,-0.093426,0.730652,0.737058,-0.204399,-0.100990,0.721133,-0.035987,0.720174
656,-0.111359,0.792717,-0.042777,-0.200866,0.177290,-0.197256,-0.157094,0.618329,0.345660,-0.204141,...,0.468026,-0.119549,0.587847,0.257681,0.609835,-0.155365,-0.202416,0.715337,-0.192799,0.378262


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.030033,-0.168047,0.190528,0.228052,-0.133099,-0.073283,0.147607,-0.080972,-0.119341,-0.105819,...,-0.170909,-0.152570,-0.167494,-0.133137,-0.155304,-0.128978,Abnormal,P42,FP-B,AB01
1,-0.072033,0.059004,0.264769,0.037914,-0.195818,-0.077529,0.233249,-0.063936,-0.203762,-0.133224,...,-0.126178,-0.110011,-0.137647,-0.091916,-0.024763,-0.119757,Abnormal,P2,FP-A,AB01
2,-0.066855,-0.098778,-0.174455,-0.097675,-0.172910,-0.119162,-0.183226,-0.079950,-0.119603,-0.213992,...,-0.155933,-0.118044,-0.116492,0.014855,-0.200639,-0.115070,Abnormal,P51,FP-C,AB01
3,-0.164899,-0.137043,0.030739,0.282006,-0.110273,-0.154812,0.096774,-0.149772,-0.065819,-0.151823,...,-0.107320,-0.144773,-0.165934,0.055076,-0.071099,-0.100872,Abnormal,P1,FP-A,AB01
4,-0.168137,-0.054375,-0.102106,0.019492,-0.163883,-0.227633,-0.084770,-0.097355,-0.082006,-0.156650,...,-0.196706,-0.237798,-0.075828,0.223701,-0.144849,-0.078928,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,-0.089932,0.966502,-0.061667,-0.077552,0.749401,-0.156538,-0.115387,0.022432,0.792956,-0.158872,...,-0.027156,-0.183324,-0.014921,0.625583,-0.066625,0.457226,Normal,P32,FP-B,Normal
654,0.041996,-0.166815,0.024205,-0.179397,-0.155007,-0.190471,-0.091025,0.342036,0.167473,-0.130458,...,0.043427,-0.186619,-0.126861,0.025019,-0.139925,-0.125518,Normal,P42,FP-B,Normal
655,-0.073415,0.694064,-0.195658,-0.050105,0.957885,-0.167023,-0.168058,0.331353,0.739299,-0.216589,...,0.737058,-0.204399,-0.100990,0.721133,-0.035987,0.720174,Normal,P52,FP-C,Normal
656,-0.111359,0.792717,-0.042777,-0.200866,0.177290,-0.197256,-0.157094,0.618329,0.345660,-0.204141,...,0.609835,-0.155365,-0.202416,0.715337,-0.192799,0.378262,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold6_1.csv')